# **Projet de big data**

## **Mise en place de l’environnement de travail**

### Installation du scikit-learn & matplotlib

In [ ]:
#install scikit-learn
 #  ! pip install -U scikit-learn

#install matplotlib
 #  ! pip install -U matplotlib

### installation du spark


Pour exécuter spark dans Colab, nous devons d'abord installer toutes les dépendances dans l'environnement Colab, c'est-à-dire Apache Spark 3.1.1 avec hadoop 2.7, Java 8 et Findspark pour localiser spark dans le système.

In [10]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [11]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz


In [12]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz


In [13]:
!pip install -q findspark

Maintenant que nous avons installé Spark et Java dans Colab, il est temps de définir le chemin d'environnement qui vnous permet d'exécuter Pyspark dans notre environnement Colab.

In [14]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [15]:
#verfication du type
os.environ["SPARK_HOME"]

'/content/spark-3.1.1-bin-hadoop2.7'

### Creation de Sparkcontext & Sparksession

In [17]:
#SparkContext
#We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "my_app")

ValueError: ignored

In [18]:
#SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("my_app")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

## **Génération de données**

In [40]:
from sklearn.datasets import make_blobs
import numpy
X,y= make_blobs(n_samples=100, centers=3, n_features=2,random_state=0)

In [49]:
Y=[]
for i in range(0,len(y)):
  k=[y[i]]
  Y.append(k)
X = numpy.append(X, Y, 1)

In [37]:
from pyspark.sql.types import Row
rdd = sc.parallelize(df)
dff = rdd.map(lambda c: Row(first_feature=float(c[0]), second_feature=float(c[1]), label=float(c[2]))).toDF()

Py4JJavaError: ignored

In [ ]:
from pyspark.sql.types import Row
dff = rdd.toDF() 